In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import csv
import cv2
import numpy as np
import random
import os

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision.transforms.functional import to_tensor

from collections import OrderedDict

In [3]:
TRAIN_PATH = "/content/drive/MyDrive/hw5/captcha-hacker/train"
TEST_PATH = "/content/drive/MyDrive/hw5/captcha-hacker/test"
device = "cuda"

In [4]:
characters = '0123456789abcdefghijklmnopqrstuvwxyz'

In [5]:
class taskData(Dataset):
  def __init__(self, data, root, digit, count):
    self.root = root
    self.data = data
    self.digit = digit
    self.count = count

  def __getitem__(self, index):
    filename, label = self.data[index]
    # turn everything to tensor
    img = to_tensor(cv2.imread(f"{self.root}/{filename}")) # the image
    encode = torch.tensor([characters.find(x) for x in label], dtype=torch.long) # turn characters to indices (target)
    digits = torch.full(size=(1, ), fill_value=self.digit, dtype=torch.long) # how many digits originally predicted (inputLength)
    counts = torch.full(size=(1, ), fill_value=self.count, dtype=torch.long) # how many digits should there be (targetLength)
    return img, encode, digits, counts 

  def __len__(self):
    return len(self.data)

In [6]:
train_data1 = []
train_data2 = []
train_data3 = []

# append the data classified by starting string
with open(f'{TRAIN_PATH}/annotations.csv', newline='') as csvfile:
  for row in csv.reader(csvfile, delimiter=','):
    if row[0].startswith("task1"):
      train_data1.append(row)
    elif row[0].startswith("task2"):
      train_data2.append(row)
    elif row[0].startswith("task3"):
      train_data3.append(row)

ds1 = taskData(train_data1, root=TRAIN_PATH, digit=3, count=1)
train1_ds, val1_ds = torch.utils.data.random_split(ds1, [1800, 200]) 
train1_dl = DataLoader(train1_ds, batch_size=300, num_workers=2, drop_last=True, shuffle=True)
val1_dl = DataLoader(val1_ds, batch_size=200, num_workers=2, drop_last=True, shuffle=True)

ds2 = taskData(train_data2, root=TRAIN_PATH, digit=4, count=2)
train2_ds, val2_ds = torch.utils.data.random_split(ds2, [2250, 250])
train2_dl = DataLoader(train2_ds, batch_size=300, num_workers=4, drop_last=True, shuffle=True)
val2_dl = DataLoader(val2_ds, batch_size=250, num_workers=4, drop_last=True, shuffle=True)

ds3 = taskData(train_data3, root=TRAIN_PATH, digit=6, count=4)
train3_ds, val3_ds = torch.utils.data.random_split(ds3, [2700, 300])
train3_dl = DataLoader(train3_ds, batch_size=300, num_workers=4, drop_last=True, shuffle=True)
val3_dl = DataLoader(val3_ds, batch_size=200, num_workers=4, drop_last=True, shuffle=True)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
class Model(nn.Module):
  def __init__(self): # the output should be limited from 0 to z
    super(Model, self).__init__()
    # channels and pools will be different in different layers
    channels = [32, 64, 128, 256, 256]
    pools = [2, 2, 2, 2, (2, 1)]
    modules = OrderedDict()
    # the block would be iterated in the sequence ((conv2d, batchnorm, relu)*2, maxpool)
    def blocks(name, in_channels, out_channels):
        modules[f'conv{name}'] = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=(1, 1))
        modules[f'batchnorm{name}'] = nn.BatchNorm2d(out_channels)
        modules[f'relu{name}'] = nn.ReLU(inplace=True)  
    channel1 = 3 # (3, 32 -> 32, 32)
    for block, (n_channel, k_pool) in enumerate(zip(channels, pools)):
        for layer in range(1, 3):
            blocks(f'{block+1}{layer}', channel1, n_channel)
            channel1 = n_channel
        modules[f'maxpool{block + 1}'] = nn.MaxPool2d(k_pool)
    # dropout at last
    modules[f'dropout'] = nn.Dropout(0.25, inplace=True)   
    self.cnn = nn.Sequential(modules)
    # lstm and the last linear layer
    self.lstm = nn.LSTM(input_size=512, hidden_size=128, num_layers=2, bidirectional=True)
    self.final = nn.Linear(in_features=256, out_features=36)

  def forward(self, x):
    x = self.cnn(x)
    x = x.reshape(x.shape[0], -1, x.shape[-1])
    x = x.permute(2, 0, 1)
    x, _ = self.lstm(x)
    x = self.final(x)
    return x

In [8]:
def p(seq, num):
  # TASK1
  if num == 1: 
    return seq[0]
  # TASK2
  elif num == 2: 
    return seq[0]+seq[3]
  # TASK3
  else: 
    s = ''.join([x for j, x in enumerate(seq[:-1]) if x != characters[0] and j != 0])
    if len(s) < 2 and seq[3] == '0' and seq[4] == '0':
      s += '0'
    if len(s) < 2 and seq[1] == '0' and seq[2] == '0':
      s = '0' + s
    s = seq[0] + s
    if len(s) == 4:
      if s[2] != s[3]:
        s = "".join(sorted(set(s), key=s.index))
      else:
        s = s[:3]
    s += seq[-1]
    return s

In [9]:
def accuracy(encode, output, num):
  outputs = output.detach().permute(1, 0, 2).argmax(dim=-1).cpu().numpy()
  encoded = encode.cpu().numpy()
  correct = 0
  total = 0
  for true, pred in zip(encoded, outputs):
    total += 1
    goal = ''.join([characters[x] for x in true]).replace(' ', '')
    prediction = ''.join([characters[x] for x in pred])
    print(goal , prediction, p(prediction, num)) # check the result
    if goal == p(prediction, num):
      correct += 1
  acc = correct / total
  return acc

In [10]:
def train(model, optimizer, dataloader):
  model.train()
  # track the process
  with tqdm(dataloader) as pbar:

    # run through all data
    for batch_index, (data, encode, digits, counts) in enumerate(pbar):
      # to cuda
      data = data.to(device)
      encode = encode.to(device)
      # predict
      pred = model(data)
      # calculate the loss
      outputLs = F.log_softmax(pred, dim=-1)
      loss = F.ctc_loss(outputLs, encode, digits, counts)
      # update
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      pbar.set_description(f'Training Progress')

In [11]:
def valid(model, optimizer, dataloader, num):
  model.eval()
  # track the process
  with tqdm(dataloader) as pbar, torch.no_grad():

    # run through all data
    for batch_index, (data, encode, digits, counts) in enumerate(pbar):
      # to cuda
      data = data.to(device)
      encode = encode.to(device)
      # predict and calculate the accuracy
      pred = model(data)
      acc = accuracy(encode, pred, num)
      
      print('Accuracy: %.3f' % acc)

In [12]:
# TASK1 lr=1e-3
model1 = Model().to(device)
optimizer = torch.optim.Adam(model1.parameters(), 1e-3, amsgrad=True)
epochs = 50
for epoch in range(1, epochs + 1):
  print("Epoch ", epoch)
  train(model1, optimizer, train1_dl)
  valid(model1, optimizer, val1_dl, 1)
  print("---------------------------------------------------------------------------")

Epoch  1


  0%|          | 0/1 [00:00<?, ?it/s]

9 0000 0
2 0000 0
8 0000 0
0 0000 0
7 0000 0
2 0000 0
4 0000 0
7 0000 0
5 0000 0
1 0000 0
5 0000 0
3 0000 0
8 0000 0
7 0000 0
8 0000 0
5 0000 0
0 0000 0
5 0000 0
4 0000 0
6 0000 0
2 0000 0
1 0000 0
0 0000 0
2 0000 0
1 0000 0
9 0000 0
3 0000 0
6 0000 0
8 0000 0
1 0000 0
6 0000 0
1 0000 0
3 0000 0
5 0000 0
5 0000 0
6 0000 0
1 0000 0
9 0000 0
0 0000 0
8 0000 0
9 0000 0
8 0000 0
2 0000 0
8 0000 0
3 0000 0
5 0000 0
8 0000 0
6 0000 0
5 0000 0
9 0000 0
1 0000 0
2 0000 0
0 0000 0
7 0000 0
1 0000 0
1 0000 0
2 0000 0
2 0000 0
8 0000 0
7 0000 0
8 0000 0
6 0000 0
3 0000 0
3 0000 0
5 0000 0
3 0000 0
4 0000 0
7 0000 0
5 0000 0
7 0000 0
1 0000 0
2 0000 0
1 0000 0
8 0000 0
2 0000 0
0 0000 0
5 0000 0
6 0000 0
9 0000 0
4 0000 0
7 0000 0
0 0000 0
2 0000 0
9 0000 0
2 0000 0
8 0000 0
9 0000 0
7 0000 0
8 0000 0
9 0000 0
5 0000 0
8 0000 0
5 0000 0
5 0000 0
4 0000 0
4 0000 0
5 0000 0
2 0000 0
3 0000 0
7 0000 0
9 0000 0
1 0000 0
0 0000 0
9 0000 0
8 0000 0
8 0000 0
3 0000 0
6 0000 0
0 0000 0
6 0000 0
9 0000 0
5

100%|██████████| 1/1 [00:41<00:00, 41.27s/it]



8 0000 0
0 0000 0
6 0000 0
0 0000 0
4 0000 0
8 0000 0
7 0000 0
8 0000 0
2 0000 0
6 0000 0
7 0000 0
Accuracy: 0.105
---------------------------------------------------------------------------
Epoch  2


...

5 5000 5
8 8000 8
2 2000 2
5 5000 5
4 4000 4
2 2000 2
6 6000 6
9 9000 9
0 0000 0
1 1000 1
0 0000 0
5 5000 5
6 7000 7
1 1000 1
7 7000 7
8 8000 8
4 4000 4
6 6000 6
8 8000 8
0 0000 0
4 4000 4
1 1000 1
9 9000 9
6 6000 6
2 2000 2
5 5000 5
1 1000 1
4 4000 4
2 2000 2
0 0000 0
2 2000 2
2 2000 2
7 7000 7
1 1000 1
8 8000 8
6 6000 6
1 1000 1
7 7000 7
7 7000 7
5 5000 5
3 3000 3
2 2000 2
1 1000 1
8 8000 8
7 7000 7
9 9000 9
5 5000 5
7 7000 7
6 6000 6
1 1000 1
8 8000 8
9 9000 9
5 5000 5
5 5000 5
0 0000 0
9 9000 9
3 3000 3
2 2000 2
7 7000 7
0 0000 0
8 8000 8
0 0000 0
5 5000 5
8 8000 8
8 8000 8
8 8000 8
1 1000 1
8 8000 8
8 8000 8
2 2000 2
2 2000 2
3 3000 3
6 6000 6
1 1000 1
9 9000 9
0 0000 0
8 8000 8
3 3000 3
2 2000 2
6 6000 6
5 5000 5
2 2000 2
5 5000 5
3 3000 3
8 8000 8
8 8000 8
0 0000 0
7 7000 7
1 1000 1
0 0000 0
3 3000 3
1 1000 1
9 9000 9
8 8000 8
1 1000 1
4 4000 4
6 6000 6
8 8000 8
2 2000 2
4 4000 4
8 8000 8
5 5000 5
4 4000 4
3 3000 3
8 8000 8
5 5000 5
6 6000 6
1 1000 1
3 3000 3
2 2000 2
6 6000 6
5

100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


 5000 5
1 1000 1
4 4000 4
2 2000 2
4 4000 4
3 3000 3
0 0000 0
9 9000 9
5 5000 5
8 8000 8
9 9000 9
Accuracy: 0.995
---------------------------------------------------------------------------
Epoch  50


100%|██████████| 1/1 [00:00<00:00,  1.35it/s]

5 5000 5
1 1000 1
5 5000 5
9 9000 9
9 9000 9
1 1000 1
8 8000 8
8 8000 8
7 7000 7
8 8000 8
2 2000 2
6 6000 6
6 6000 6
2 2000 2
6 6000 6
8 8000 8
2 2000 2
6 6000 6
7 7000 7
6 7000 7
8 8000 8
1 1000 1
4 4000 4
6 6000 6
5 5000 5
5 5000 5
2 2000 2
6 6000 6
3 3000 3
0 0000 0
1 1000 1
8 8000 8
5 5000 5
4 4000 4
6 6000 6
6 6000 6
4 4000 4
5 5000 5
5 5000 5
0 0000 0
0 0000 0
7 7000 7
8 8000 8
2 2000 2
4 4000 4
5 5000 5
4 4000 4
9 9000 9
6 6000 6
8 8000 8
5 5000 5
5 5000 5
4 4000 4
1 1000 1
2 2000 2
7 7000 7
4 4000 4
1 1000 1
2 2000 2
8 8000 8
9 9000 9
0 0000 0
3 3000 3
1 1000 1
6 6000 6
3 3000 3
5 5000 5
8 8000 8
2 2000 2
1 1000 1
7 7000 7
7 7000 7
2 2000 2
9 9000 9
1 1000 1
4 4000 4
6 6000 6
0 0000 0
5 5000 5
3 3000 3
2 2000 2
9 9000 9
1 1000 1
3 3000 3
7 7000 7
8 8000 8
1 1000 1
9 9000 9
5 5000 5
5 5000 5
7 7000 7
7 7000 7
8 8000 8
6 6000 6
9 9000 9
3 3000 3
4 4000 4
3 3000 3
7 7000 7
0 0000 0
8 8000 8
0 0000 0
2 2000 2
9 9000 9
0 0000 0
4 4000 4
9 9000 9
2 2000 2
2 2000 2
0 0000 0
6 6000 6
0

In [13]:
# lr=1e-4
optimizer = torch.optim.Adam(model1.parameters(), 1e-4, amsgrad=True)
epochs = 30
for epoch in range(1, epochs + 1):
  print("Epoch ", epoch)
  train(model1, optimizer, train1_dl)
  valid(model1, optimizer, val1_dl, 1)
  print("---------------------------------------------------------------------------")

Epoch  1


  0%|          | 0/1 [00:00<?, ?it/s]

1 1000 1
3 3000 3
0 0000 0
0 0000 0
9 9000 9
7 7000 7
3 3000 3
2 2000 2
9 9000 9
8 8000 8
2 2000 2
3 3000 3
3 3000 3
6 6000 6
5 5000 5
3 3000 3
2 2000 2
3 3000 3
0 0000 0
9 9000 9
1 1000 1
2 2000 2
3 3000 3
5 5000 5
3 3000 3
1 1000 1
0 0000 0
2 2000 2
5 5000 5
0 0000 0
7 7000 7
8 8000 8
9 9000 9
4 4000 4
8 8000 8
0 0000 0
0 0000 0
8 8000 8
2 2000 2
3 3000 3
1 1000 1
6 6000 6
1 1000 1
8 8000 8
3 3000 3
4 4000 4
8 8000 8
6 4000 4
8 8000 8
0 0000 0
9 9000 9
1 1000 1
6 6000 6
5 5000 5
1 1000 1
2 2000 2
7 7000 7
0 0000 0
5 5000 5
1 1000 1
5 5000 5
8 8000 8
4 4000 4
7 7000 7
4 4000 4
8 8000 8
3 3000 3
6 6000 6
0 0000 0
8 8000 8
3 3000 3
5 5000 5
1 1000 1
3 3000 3
8 8000 8
9 9000 9
0 0000 0
0 0000 0
9 9000 9
8 8000 8
2 2000 2
0 0000 0
9 9000 9
5 5000 5
8 8000 8
4 4000 4
9 9000 9
5 5000 5
6 6000 6
6 6000 6
8 8000 8
5 5000 5
9 9000 9
7 7000 7
7 7000 7
8 8000 8
1 1000 1
2 2000 2
6 6000 6
5 5000 5
1 1000 1
0 0000 0
8 8000 8
1 1000 1
0 0000 0
5 5000 5
1 1000 1
5 5000 5
2 2000 2
0 0000 0
5 5000 5
9

100%|██████████| 1/1 [00:00<00:00,  1.20it/s]



3 3000 3
8 8000 8
Accuracy: 0.995
---------------------------------------------------------------------------
Epoch  2


...

 6
6 6000 6
9 9000 9
5 5000 5
7 7000 7
5 5000 5
5 5000 5
2 2000 2
9 9000 9
8 8000 8
9 9000 9
3 3000 3
2 2000 2
6 6000 6
Accuracy: 0.995
---------------------------------------------------------------------------
Epoch  30


  0%|          | 0/1 [00:00<?, ?it/s]

4 4000 4
9 9000 9
1 1000 1
8 8000 8
9 9000 9
5 5000 5
5 5000 5
1 1000 1
5 5000 5
7 7000 7
1 1000 1
3 3000 3
7 7000 7
8 8000 8
8 8000 8
6 6000 6
4 4000 4
1 1000 1
8 8000 8
1 1000 1
3 3000 3
1 1000 1
7 7000 7
4 4000 4
6 6000 6
0 0000 0
0 0000 0
3 3000 3
5 5000 5
3 3000 3
9 9000 9
0 0000 0
5 5000 5
7 7000 7
1 1000 1
7 7000 7
5 5000 5
5 5000 5
8 8000 8
4 4000 4
0 0000 0
4 4000 4
1 1000 1
2 2000 2
4 4000 4
8 8000 8
6 6000 6
6 6000 6
3 3000 3
2 2000 2
8 8000 8
4 4000 4
2 2000 2
8 8000 8
2 2000 2
5 5000 5
0 0000 0
2 2000 2
3 3000 3
0 0000 0
2 2000 2
8 8000 8
1 1000 1
2 2000 2
1 1000 1
8 8000 8
8 8000 8
1 1000 1
3 3000 3
0 0000 0
4 4000 4
1 1000 1
1 1000 1
9 9000 9
0 0000 0
4 4000 4
2 2000 2
6 6000 6
9 9000 9
1 1000 1
3 3000 3
0 0000 0
2 2000 2
2 2000 2
2 2000 2
0 0000 0
5 5000 5
5 5000 5
7 7000 7
9 9000 9
5 5000 5
4 4000 4
0 0000 0
8 8000 8
3 3000 3
9 9000 9
3 3000 3
8 8000 8
7 7000 7
7 7000 7
7 7000 7
7 7000 7
6 7000 7
1 1000 1
3 3000 3
0 0000 0
2 2000 2
6 6000 6
6 6000 6
4 4000 4
4 4000 4
0

100%|██████████| 1/1 [00:00<00:00,  1.24it/s]

3000 3
0 0000 0
3 3000 3
6 6000 6
9 9000 9
5 5000 5
5 5000 5
7 7000 7
6 6000 6
9 9000 9
Accuracy: 0.995
---------------------------------------------------------------------------


In [15]:
torch.save(model1, 'model1.pth')
!cp model1.pth ./drive/MyDrive/hw5/captcha-hacker/model1.pth 

In [16]:
# TASK2 lr=1e-3
model2 = Model().to(device)
optimizer = torch.optim.Adam(model2.parameters(), 1e-3, amsgrad=True)
epochs = 60
for epoch in range(1, epochs + 1):
  print("Epoch ", epoch)
  train(model2, optimizer, train2_dl)
  valid(model2, optimizer, val2_dl, 2)


Epoch  1


100%|██████████| 1/1 [00:49<00:00, 49.16s/it]

cs 0000 00
np 0000 00
t6 0000 00
p8 0000 00
7c 0000 00
mv 0000 00
km 0000 00
69 0000 00
me 0000 00
sx 0000 00
2f 0000 00
hf 0000 00
fh 0000 00
cr 0000 00
ph 0000 00
k8 0000 00
85 0000 00
c5 0000 00
px 0000 00
wd 0000 00
05 0000 00
dj 0000 00
vs 0000 00
wn 0000 00
82 0000 00
yx 0000 00
fp 0000 00
7e 0000 00
9f 0000 00
1x 0000 00
3c 0000 00
09 0000 00
49 0000 00
n5 0000 00
dp 0000 00
v3 0000 00
kd 0000 00
d6 0000 00
2e 0000 00
y4 0000 00
j6 0000 00
p9 0000 00
3s 0000 00
ws 0000 00
mh 0000 00
sf 0000 00
dc 0000 00
9e 0000 00
j6 0000 00
fs 0000 00
c9 0000 00
74 0000 00
c8 0000 00
xr 0000 00
cw 0000 00
3h 0000 00
7n 0000 00
72 0000 00
w5 0000 00
n0 0000 00
ty 0000 00
my 0000 00
wn 0000 00
pf 0000 00
km 0000 00
4e 0000 00
0k 0000 00
ax 0000 00
9h 0000 00
ws 0000 00
yy 0000 00
sj 0000 00
5r 0000 00
v8 0000 00
a2 0000 00
kx 0000 00
w9 0000 00
94 0000 00
vx 0000 00
86 0000 00
ha 0000 00
2n 0000 00
0x 0000 00
sy 0000 00
ay 0000 00
wy 0000 00
2c 0000 00
td 0000 00
xe 0000 00
ap 0000 00
jv 0000 00

100%|██████████| 1/1 [00:49<00:00, 49.37s/it]


Epoch  2


...

 c9
8k 800k 8k
t1 t001 t1
nx n00x nx
s0 s000 s0
nx n00x nx
hv h00v hv
km k00m km
n7 n007 n7
2r 200r 2r
85 8005 85
ne n00e ne
c9 c009 c9
ea e00a ea
aj a00j aj
cr c00r cr
7m 700m 7m
yc y00c yc
d9 d009 d9
yx y00x yx
x2 x002 x2
fp f00p fp
mk m00k mk
2n 200n 2n
69 6009 69
j6 j006 j6
97 9007 97
t6 t006 t6
2w 200w 2w
ws w00s ws
fs f00s fs
fs f00s fs
cs c00s cs
t9 t009 t9
5r 500r 5r
a2 a002 a2
2y 200y 2y
ax a00x ax
3c 300c 3c
2k 200k 2k
j6 j006 j6
k8 k008 k8
0m 000m 0m
me m00e me
02 0002 02
x3 x003 x3
kd k00d kd
xh x00h xh
7e 700e 7e
fh 500d 5d
h6 h006 h6
ee e00e ee
xr x00r xr
0w 000w 0w
2c 200c 2c
rw r00w rw
p1 p001 p1
ny n00y ny
2c 200c 2c
78 7008 78
7s 700s 7s
9f 900f 9f
a9 a009 a9
sf s00f sf
dp d00p dp
wm w00m wm
cw c00w cw
9h 900h 9h
Accuracy: 0.976
Epoch  60


  0%|          | 0/1 [00:00<?, ?it/s]

2c 200c 2c
2c 200c 2c
2p 200p 2p
y7 y007 y7
wm w00m wm
74 7004 74
1x 100x 1x
rr r00r rr
85 8005 85
t1 t001 t1
e4 e004 e4
5n 500n 5n
53 5003 53
9n 900n 9n
j6 j006 j6
01 0001 01
mj m00j mj
3c 300c 3c
4e 400e 4e
2f 200f 2f
1c 100c 1c
hf h00f hf
t6 t006 t6
ne n00e ne
j6 j006 j6
yx y00x yx
6c 600c 6c
3d 300d 3d
c8 c008 c8
pr p00r pr
2c 200c 2c
72 7002 72
3s 300s 3s
y4 y004 y4
km k00m km
jx j00x jx
nx n00x nx
aj a00j aj
a1 a001 a1
ha h00a ha
7e 700e 7e
n7 n007 n7
wm w00m wm
kx k00x kx
7s 700s 7s
x3 x003 x3
ax a00x ax
s0 s000 s0
md m00d md
n7 n007 n7
cw c00w cw
2n 200n 2n
ax a00x ax
ch c00h ch
kx k00x kx
je j00e je
fh 100h 1h
km k00m km
1r 100r 1r
ea e00a ea
k3 k003 k3
wj w00j wj
d9 d009 d9
v3 v003 v3
2e 200e 2e
0w 000w 0w
td t00d td
cr c00r cr
vy v00y vy
r0 r000 r0
e7 e007 e7
7x 700x 7x
r5 r005 r5
wh w00h wh
8k 800k 8k
my m00y my
9a 900a 9a
f5 f005 f5
ap a00p ap
dt d00t dt
61 6001 61
yy y00y yy
9s 900s 9s
9f 900f 9f
02 0002 02
e3 e003 e3
0m 000m 0m
hv h00v hv
n5 n005 n5
sf s00f sf
h3 h003 h3

100%|██████████| 1/1 [00:01<00:00,  1.06s/it]

 w00s ws
vx v00x vx
ej e00j ej
32 3002 32
w5 w005 w5
19 1009 19
hr h00r hr
e4 e004 e4
mk m00k mk
7n 700n 7n
69 6009 69
mh m00h mh
np n00p np
53 5003 53
xr x00r xr
wd w00d wd
ed e00d ed
t7 t007 t7
ny n00y ny
sx s00x sx
p4 p004 p4
2y 200y 2y
ph p00h ph
j9 j009 j9
k8 k008 k8
wn w00n wn
ee e00e ee
78 7008 78
t1 t001 t1
jv j00v jv
s7 s007 s7
4r 400r 4r
fk f00k fk
kd k00d kd
ve v00e ve
np n00p np
9s 900s 9s
kt k00c kc
rw r00w rw
t6 t006 t6
72 7002 72
s5 s005 s5
tt t00t tt
dj d00j dj
x2 x002 x2
1d 100d 1d
nf n00f nf
0k n00k nk
xe x00e xe
c3 c003 c3
7r 700r 7r
ek e00k ek
yr y00r yr
xc x00c xc
j6 j006 j6
3c 300c 3c
x6 x006 x6
nr n00r nr
k8 k008 k8
c5 c005 c5
t9 t009 t9
am a00m am
vk v00k vk
me m00e me
3s 300s 3s
xc x00c xc
d6 d006 d6
1a 100a 1a
86 8006 86
aa a00a aa
8f 800f 8f
a9 a009 a9
mv m00v mv
Accuracy: 0.976


In [17]:
# lr=1e-4
optimizer = torch.optim.Adam(model2.parameters(), 1e-4, amsgrad=True)
epochs = 40
for epoch in range(1, epochs + 1):
  print("Epoch ", epoch)
  train(model2, optimizer, train2_dl)
  valid(model2, optimizer, val2_dl, 2)

Epoch  1


100%|██████████| 1/1 [00:00<00:00,  1.25it/s]

wm w00m wm
9f 900y 9y
ws w00s ws
y7 y007 y7
ea e00a ea
pa p00a pa
1c 100c 1c
nx n00x nx
sj s00j sj
np n00p np
yy y00y yy
cd c00d cd
pt p00t pt
4r 400r 4r
aj a00j aj
rn r00n rn
vs v00s vs
5n 500n 5n
09 0009 09
v3 v003 v3
05 0005 05
9e 900e 9e
97 9007 97
kx k00x kx
7n 100n 1n
32 3002 32
a9 a009 a9
w5 w005 w5
k3 k003 k3
2r 200r 2r
aa a00a aa
cw c00w cw
th t00h th
md m00d md
x3 x003 x3
x8 x008 x8
53 5003 53
yc y00c yc
dp d00p dp
fh 500d 5d
2c 200c 2c
w9 w009 w9
7c 700c 7c
e3 e003 e3
xr x00r xr
ny n00y ny
yr y00r yr
mv m00v mv
mv m00v mv
c8 c008 c8
9n 900n 9n
d6 d006 d6
v8 k008 k8
rw r00w rw
sj s00j sj
cs c00s cs
5a 500a 5a
ed e00d ed
vx v00x vx
f4 f004 f4
48 4008 48
p4 p004 p4
am a00m am
78 7008 78
pf p00f pf
km k00m km
e4 e004 e4
3s 300s 3s
01 0001 01
c9 c009 c9
8k 800k 8k
t6 t006 t6
vx v00x vx
2k 200k 2k
ss s00s ss
9f 900f 9f
fh 100h 1h
d9 d009 d9
nx n00x nx
n0 n000 n0
c3 c003 c3
sd s00d sd
kd k00d kd
0x 000x 0x
sf s00f sf
wn w00n wn
wh w00h wh
2w 200w 2w
8k 800k 8k
rw r00w rw
ax a00x ax

100%|██████████| 1/1 [00:00<00:00,  1.08it/s]


Epoch  2


...

Epoch  40


  0%|          | 0/1 [00:00<?, ?it/s]

mv m00v mv
th t00h th
49 4009 49
7e 700e 7e
fh f00h fh
0k n00k nk
t9 t009 t9
xc x00c xc
je j00e je
7c 700c 7c
8k 800k 8k
9f 9001 91
ee e00e ee
2c 200c 2c
r5 r005 r5
np n00p np
aj a00j aj
48 4008 48
3s 300s 3s
wy w00y wy
1d 100d 1d
ss s00s ss
wh w00h wh
wn w00n wn
xc x00c xc
s0 s000 s0
a1 a001 a1
hv h00v hv
td t00d td
e3 e003 e3
y4 y004 y4
ay a00y ay
n7 n007 n7
xe x00e xe
p9 p009 p9
c3 c003 c3
xh x00h xh
pt p00t pt
td t00d td
cs c00s cs
1c 100c 1c
r0 r000 r0
d9 d009 d9
sy s00y sy
sj s00j sj
vx v00x vx
cd c00d cd
k8 k008 k8
hf h00f hf
rr r00r rr
mv m00v mv
kx k00x kx
km k00m km
fp f00p fp
e4 e004 e4
ph p00h ph
ax a00x ax
4r 400r 4r
3d 300d 3d
vx v00x vx
1c 100c 1c
km k00m km
x3 x003 x3
2y 200y 2y
k0 k000 k0
k3 k003 k3
82 8002 82
jx j00x jx
jv j00v jv
j6 j006 j6
9f 900f 9f
vs v00s vs
7n 700n 7n
mj m00j mj
69 6009 69
2e 200e 2e
cw c00w cw
fk f00k fk
kt k00c kc
8f 800f 8f
7n 700n 7n
rn r00n rn
8k 800k 8k
n7 n007 n7
fs f00s fs
9e 900e 9e
dp d00p dp
wm w00m wm
h6 h006 h6
97 9007 97
19 1009 19

100%|██████████| 1/1 [00:01<00:00,  1.09s/it]

fs f00s fs
n5 n005 n5
ak a00k ak
09 0009 09
78 7008 78
p4 p004 p4
3c 300c 3c
74 7004 74
nf n00f nf
ch c00h ch
pr p00r pr
9n 900n 9n
t6 t006 t6
94 9004 94
yy y00y yy
vk v00k vk
w5 w005 w5
t7 t007 t7
sx s00x sx
tt t00t tt
ha h00a ha
a9 a009 a9
0m 000m 0m
9w 900w 9w
58 5008 58
72 7002 72
w9 w009 w9
2r 200r 2r
ty t00y ty
dj d00j dj
sf s00f sf
dt d00t dt
f5 f005 f5
72 7002 72
6r 600r 6r
s5 s005 s5
6c 600c 6c
53 5003 53
69 6009 69
53 5003 53
j9 j009 j9
4e 400e 4e
3c 300c 3c
ap a00p ap
1a 100a 1a
85 8005 85
wm w00m wm
am a00m am
x2 x002 x2
mh m00h mh
nr n00r nr
ny n00y ny
pa p00a pa
01 0001 01
s7 s007 s7
ve v00e ve
t1 t001 t1
ed e00d ed
1x 100x 1x
c9 c009 c9
5a 500a 5a
3s 300s 3s
aa a00a aa
wj w00j wj
86 8006 86
Accuracy: 0.984


In [19]:
# lr=1e-5
optimizer = torch.optim.Adam(model2.parameters(), 1e-5, amsgrad=True)
epochs = 30
for epoch in range(1, epochs + 1):
  print("Epoch ", epoch)
  train(model2, optimizer, train2_dl)
  valid(model2, optimizer, val2_dl, 2)

Epoch  1


  0%|          | 0/1 [00:00<?, ?it/s]

58 5008 58
ve v00e ve
kt k00c kc
jv j00v jv
d9 d009 d9
r0 r000 r0
x6 x006 x6
9h 900h 9h
v8 v008 v8
yy y00y yy
cd c00d cd
9a 900a 9a
sd s00d sd
69 6009 69
wj w00j wj
05 0005 05
hv h00v hv
74 7004 74
0x 000x 0x
s7 s007 s7
c3 c003 c3
1d 100d 1d
9f 900f 9f
78 7008 78
n7 n007 n7
wh w00h wh
sr s00r sr
vk v00k vk
3s 300s 3s
86 8006 86
9s 900s 9s
3h 300h 3h
0w 000w 0w
mk m00k mk
je j00e je
7n 700n 7n
3c 300c 3c
aj a00j aj
fs f00s fs
ap a00p ap
ss s00s ss
f5 f005 f5
w5 w005 w5
c9 c009 c9
2w 200w 2w
2y 200y 2y
x3 x003 x3
a1 a001 a1
yc y00c yc
h3 h003 h3
t9 t009 t9
dj d00j dj
n7 n007 n7
x8 x008 x8
92 9002 92
k3 k003 k3
mv m00v mv
1r 100r 1r
t1 t001 t1
mv m00v mv
49 4009 49
f4 f004 f4
mv m00v mv
ny n00y ny
my m00y my
t7 t007 t7
xh x00h xh
ne n00e ne
k8 k008 k8
4e 400e 4e
mh m00h mh
y4 y004 y4
fk f00k fk
s5 s005 s5
td t00d td
9s 900s 9s
hf h00f hf
vs v00s vs
mh m00h mh
2c 200c 2c
2e 200e 2e
dp d00p dp
hr h00r hr
02 0002 02
6r 600r 6r
7m 700m 7m
ha h00a ha
j6 j006 j6
7s 700s 7s
8k 800k 8k
72 7002 72

100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


 td
ax a00x ax
wm w00m wm
wn w00n wn
j6 j006 j6
c8 c008 c8
rr r00r rr
n0 n000 n0
7e 700e 7e
kx k00x kx
rw r00w rw
sy s00y sy
3d 300d 3d
2n 200n 2n
ak a00k ak
rn r00n rn
9n 900n 9n
ee e00e ee
ty t00y ty
vy v00y vy
ax a00x ax
1c 100c 1c
pa p00a pa
19 1009 19
c9 c009 c9
p1 p001 p1
wm w00m wm
97 9007 97
yr y00r yr
5n 500n 5n
01 0001 01
cw c00w cw
94 9004 94
ms m00s ms
1a 100a 1a
dc d00c dc
e3 e003 e3
2e 200e 2e
p8 p008 p8
aa a00a aa
s8 s008 s8
sv s00v sv
2c 200c 2c
sx s00x sx
pr p00r pr
ay a00y ay
sj s00j sj
th t00h th
t6 t006 t6
2r 200r 2r
72 7002 72
0k n00k nk
v3 v003 v3
am a00m am
ek e00k ek
ed e00d ed
5r 500r 5r
yx y00x yx
9e 900e 9e
c5 c005 c5
dw d00w dw
vx v00x vx
cr c00r cr
61 6001 61
2k 200k 2k
p4 p004 p4
p8 p008 p8
rw r00w rw
Accuracy: 0.984
Epoch  2


...

4e 400e 4e
ph p00h ph
x2 x002 x2
2e 200e 2e
7n 700n 7n
n0 n000 n0
s8 s008 s8
v3 v003 v3
2e 200e 2e
2n 200n 2n
ha h00a ha
7s 700s 7s
wh w00h wh
md m00d md
aa a00a aa
xh x00h xh
ax a00x ax
3s 300s 3s
cw c00w cw
np n00p np
s0 s000 s0
9f 9001 91
ee e00e ee
sr s00r sr
92 9002 92
hf h00f hf
kt k00c kc
wd w00d wd
t7 t007 t7
9n 900n 9n
xc x00c xc
sx s00x sx
sj s00j sj
kx k00x kx
3d 300d 3d
xc x00c xc
2p 200p 2p
mv m00v mv
np n00p np
ve v00e ve
2c 200c 2c
y2 y002 y2
cs c00s cs
tt t00t tt
0k n00k nk
wj w00j wj
hv h00v hv
c3 c003 c3
69 6009 69
09 0009 09
pa p00a pa
y4 y004 y4
c9 c009 c9
0x 000x 0x
r5 r005 r5
53 5003 53
fk f00k fk
x6 x006 x6
th t00h th
97 9007 97
c8 c008 c8
6r 600r 6r
Accuracy: 0.984
Epoch  30


  0%|          | 0/1 [00:00<?, ?it/s]

wh w00h wh
km k00m km
k0 k000 k0
ek e00k ek
a1 a001 a1
92 9002 92
c5 c005 c5
yc y00c yc
nx n00x nx
p8 p008 p8
rr r00r rr
j6 j006 j6
8f 800f 8f
x8 x008 x8
82 8002 82
hf h00f hf
ha h00a ha
a2 a002 a2
s0 s000 s0
x6 x006 x6
md m00d md
c9 c009 c9
sd s00d sd
2w 200w 2w
fh 100h 1h
p4 p004 p4
px p00x px
5r 500r 5r
3k 300k 3k
mv m00v mv
5n 500n 5n
ej e00j ej
69 6009 69
rn r00n rn
fk f00k fk
hr h00r hr
am a00m am
ch c00h ch
02 0002 02
e4 e004 e4
sf s00f sf
7s 700s 7s
xe x00e xe
k8 k008 k8
6r 600r 6r
9s 900s 9s
xh x00h xh
p1 p001 p1
d9 d009 d9
jv j00v jv
hv h00v hv
p8 p008 p8
49 4009 49
0m 000m 0m
ax a00x ax
1a 100a 1a
86 8006 86
01 0001 01
n7 n007 n7
w5 w005 w5
09 0009 09
v3 v003 v3
yx y00x yx
9s 900s 9s
9e 900e 9e
vx v00x vx
mj m00j mj
ap a00p ap
3s 300s 3s
7n 700n 7n
ay a00y ay
xc x00c xc
6y 600y 6y
jx j00x jx
mh m00h mh
xc x00c xc
2k 200k 2k
0w 000w 0w
48 4008 48
9f 900f 9f
vk v00k vk
74 7004 74
3d 300d 3d
f5 f005 f5
4e 400e 4e
dt d00t dt
cw c00w cw
ms m00s ms
53 5003 53
my m00y my
e3 e003 e3

100%|██████████| 1/1 [00:01<00:00,  1.13s/it]

1r
4r 400r 4r
nx n00x nx
kx k00x kx
9h 900h 9h
ws w00s ws
sj s00j sj
32 3002 32
pr p00r pr
np n00p np
wn w00n wn
x3 x003 x3
wj w00j wj
ve v00e ve
19 1009 19
c8 c008 c8
72 7002 72
7c 700c 7c
8k 800k 8k
e4 e004 e4
2y 200y 2y
f4 f004 f4
kx k00x kx
aj a00j aj
r5 r005 r5
ed e00d ed
2p 200p 2p
h3 h003 h3
7e 700e 7e
dc d00c dc
sy s00y sy
t1 t001 t1
6c 600c 6c
cs c00s cs
2f 200f 2f
yr y00r yr
dp d00p dp
d6 d006 d6
wd w00d wd
5a 500a 5a
dj d00j dj
9w 900w 9w
nf n00f nf
53 5003 53
ph p00h ph
t9 t009 t9
0k n00k nk
05 0005 05
td t00d td
k8 k008 k8
t6 t006 t6
sj s00j sj
ty t00y ty
85 8005 85
pf p00f pf
sx s00x sx
fs f00s fs
fp f00p fp
n5 n005 n5
1x 100x 1x
Accuracy: 0.984


In [20]:
torch.save(model2, 'model2.pth')
!cp model2.pth ./drive/MyDrive/hw5/captcha-hacker/model2.pth 

In [12]:
# TASK3 lr=1e-3
model3 = Model().to(device)
optimizer = torch.optim.Adam(model3.parameters(), 1e-3, amsgrad=True)
epochs = 60
for epoch in range(1, epochs + 1):
  print("Epoch ", epoch)
  train(model3, optimizer, train3_dl)
  valid(model3, optimizer, val3_dl, 3)

Epoch  1


  0%|          | 0/1 [00:00<?, ?it/s]

yhrr 000000 0000
ca3h 000000 0000
y00h 000000 0000
t8cj 000000 0000
4ejs 000000 0000
8j70 000000 0000
nrdm 000000 0000
ppea 000000 0000
msva 000000 0000
a9dv 000000 0000
a587 000000 0000
ydkd 000000 0000
dwxs 000000 0000
a4n6 000000 0000
jwfd 000000 0000
an23 000000 0000
caa9 000000 0000
3v26 000000 0000
06sv 000000 0000
fsmk 000000 0000
cvtv 000000 0000
a932 000000 0000
ykrh 000000 0000
k29e 000000 0000
8hcr 000000 0000
fh02 000000 0000
5x3s 000000 0000
6028 000000 0000
asek 000000 0000
6649 000000 0000
3p41 000000 0000
7pae 000000 0000
3fp6 000000 0000
f01k 000000 0000
k416 000000 0000
ma8y 000000 0000
ejra 000000 0000
d6rh 000000 0000
8dwf 000000 0000
2rtw 000000 0000
a316 000000 0000
c4v2 000000 0000
nj5k 000000 0000
3mh6 000000 0000
v4jv 000000 0000
xjd5 000000 0000
k966 000000 0000
t6ph 000000 0000
5cee 000000 0000
hk14 000000 0000
0n0w 000000 0000
md4v 000000 0000
2511 000000 0000
mp6d 000000 0000
rwa9 000000 0000
vc93 000000 0000
p8hd 000000 0000
2n7y 000000 0000
jcsm 000000 00

100%|██████████| 1/1 [01:29<00:00, 89.24s/it]


Accuracy: 0.000
Epoch  2


...

100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


Epoch  60


100%|██████████| 1/1 [00:00<00:00,  1.21it/s]

k0ha k00h0a k0ha
03t3 003t03 03t3
y1hd y01h0d y1hd
6028 600208 6028
1t0j 10t00j 1t0j
tyxj t0yx0j tyxj
ppea p0ppna pna
dsfj d0sf0j dsfj
awac a0wa0c awac
f01k f0001k f01k
63th 60310h 631h
8j70 80j700 8j70
fnt4 f0nnt4 fnt4
ppt5 p0pt05 ppt5
ev60 e0v600 ev60
a4n6 a04n06 a4n6
4mnh 40mn0h 4mnh
16yh 106y0h 16yh
m6ht m06h0t m6ht
k29e k0290e k29e
nt3n n0t30n nt3n
06sv 006s0v 06sv
k416 k04106 k416
v900 v09000 v900
77hv 707h0v 77hv
166x 166c0x 16cx
t6x2 t06x02 t6x2
m9h7 m90h07 m9h7
724x 70244x 724x
raxe r0axxe raxe
161k 10670k 167k
4y3h 40330h 433h
mp7n m0p70n mp7n
jff4 j0ff04 jff4
71y3 701y03 71y3
e15m e0150m e15m
cyhh c0yhhh cyhh
crtd c0rt0d crtd
ypcn y0p60n yp6n
3fp6 30fp06 3fp6
vp01 v0p001 vp01
ytxy y0tx0y ytxy
6201 602001 6201
83k8 80fk08 8fk8
csdv c0sd0v csdv
4ejs 40250s 425s
96vd 966v0d 96vd
5x3s s0x30s sx3s
ce48 ce0408 ce48
e767 e07607 e767
mvkm m0vk0m mvkm
h8nk k080nk k8nk
me9d m0e09d me9d
3ee7 30eee0 3ee0
c4v2 c0m402 cm42
4f0a 40f00a 4f0a
dw1c dw010c dw1c
py9j py090j py9j
wsky w0sk0y wsk

100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


In [14]:
# lr=1e-4
optimizer = torch.optim.Adam(model3.parameters(), 1e-4, amsgrad=True)
epochs = 40
for epoch in range(1, epochs + 1):
  print("Epoch ", epoch)
  train(model3, optimizer, train3_dl)
  valid(model3, optimizer, val3_dl, 3)

Epoch  1


  0%|          | 0/1 [00:00<?, ?it/s]

2a17 20ar07 2ar7
a932 a99302 a932
nv26 n0v206 nv26
4vae 40va0e 4vae
3p41 3p0401 3p41
5nyf 5n0y0f 5nyf
3fp6 30fp06 3fp6
ppt5 p0pt05 ppt5
y00h y0000h y00h
thd9 t00d09 t0d9
hmpf h0mp0f hmpf
4fyy 40fy0y 4fyy
1t0j 10t00j 1t0j
f772 f07702 f772
3dm1 30dm01 3dm1
2rx9 20rx09 2rx9
v31x v0f10x vf1x
50fv 500f0v 50fv
v900 v09000 v900
k416 k04106 k416
8fwj 80f0wj 8fwj
3r1a 3r010a 3r1a
hxvt h0xk0t hxkt
md4v m0d40v md4v
r3tc r03t0c r3tc
n31c n0030c n03c
8hcr 8h0c0r 8hcr
w5an w05a0n w5an
v3s7 v03s07 v3s7
nrdm n0rd0w nrdw
6wh3 60whh3 6wh3
mvkm m0vk0m mvkm
5cp0 50cp00 5cp0
yt68 y0t608 yt68
2rtw 201r0w 21rw
f68j f0680j f68j
vpmk v0pmmk vpmk
a316 a03706 a376
346e 30460e 346e
hd2k h0d20k hd2k
m9h7 m90h07 m9h7
x4jt x04j0t x4jt
ejra e0jr0a ejra
ecy3 e0cx03 ecx3
71y3 701x03 71x3
njxs n0jx0s njxs
8dwf 80dw0f 8dwf
enkn e0nk0n enkn
2mn5 200n05 20n5
ykrh y0kr0h ykrh
me9d m0e09d me9d
cead c0ea0d cead
jcsm j0cs0m jcsm
kas7 k0as07 kas7
e26k e0260k e26k
2f9n 2f090n 2f9n
1x0f 1x000f 1x0f
8y9a n0y90a ny9a
py9j py090j py

100%|██████████| 1/1 [00:00<00:00,  1.02it/s]


Accuracy: 0.800
Epoch  2


...

100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


03t3 003t03 03t3
16yh 106y0h 16yh
6wh3 60whh3 6wh3
xkyc x0kk0c xkkc
6649 606409 6649
h8nk k080nk k8nk
wty4 w0ty04 wty4
4mnh 40mn0h 4mnh
k966 k99106 k916
50nj 500n0j 50nj
n31c n0030c n03c
t6x2 t06x02 t6x2
1yka 10yk0a 1yka
v900 v09000 v900
ca3h ca030h ca3h
a932 a99302 a932
161k 10670k 167k
c902 c0r002 cr02
Accuracy: 0.790
Epoch  40


  0%|          | 0/1 [00:00<?, ?it/s]

m8p4 m08p04 m8p4
ax80 a0x800 ax80
k1st k01s0t k1st
2fsf 20fs0f 2fsf
vve6 v0ve06 vve6
awac a0wa0c awac
v13a v0030a v03a
csdv c0sd0v csdv
7x6f 70x60f 7x6f
nsxc n0sx0c nsxc
hxvt h0xk0c hxkc
v4jv v04j0v v4jv
kywv k0yw0v kywv
m6ht m06h0t m6ht
0enk 00e80k 0e8k
98vs 908v0s 98vs
70e2 700e02 70e2
w38d w0380d w38d
nt3n n0t30n nt3n
3fp6 30fp06 3fp6
an23 a00203 a023
x4jt x04j0t x4jt
px01 px0001 px01
5nyf 5n0y0f 5nyf
rkym r0ky0m rkym
raxe r0axye raxye
w3dx w03d0x w3dx
y1dv y07d0v y7dv
8hcr 8h0c0r 8hcr
h8nk h080nk h8nk
n0fn n00f0n n0fn
5cee 5c0e0e 5cee
mp6d m0p60d mp6d
dw1c dw010c dw1c
jcsm j0cs0m jcsm
24c7 204c07 24c7
hmpf h0mp0f hmpf
hpst k0pc0t kpct
3x91 30x901 3x91
8xv3 8x0v03 8xv3
r32t r0320t r32t
m10w m0100w m10w
ma8y m0a80y ma8y
yt68 y0t608 yt68
30vc 300v0c 30vc
4ejs 40250s 425s
x4y1 x04y0t x4yt
0cmf 00cm0f 0cmf
f01k f0001k f01k
vpmk v0pmmk vpmk
xkyc x0kk0c xkkc
ejra e0jr0a ejra
thd9 th0d09 thd9
me9d m0e09d me9d
8vwh 8v0w0h 8vwh
jff4 j0ff04 jff4
0ekm 00ee0m 0eem
jttn j0t60n jt6n
nxsk n0xs0k n

100%|██████████| 1/1 [00:00<00:00,  1.03it/s]

In [15]:
# lr=1e-5
optimizer = torch.optim.Adam(model3.parameters(), 1e-5, amsgrad=True)
epochs = 50
for epoch in range(1, epochs + 1):
  print("Epoch ", epoch)
  train(model3, optimizer, train3_dl)
  valid(model3, optimizer, val3_dl, 3)

Epoch  1


  0%|          | 0/1 [00:00<?, ?it/s]

66e2 606e02 66e2
yt68 y0t608 yt68
h8nk k080nk k8nk
v900 v09000 v900
yv09 ypv009 ypv9
hk14 h0k704 hk74
x4jt x04j0t x4jt
77e0 700e00 70e0
16yh 106y0h 16yh
8hcr 8h0c0r 8hcr
an23 a00203 a023
m10w m0100w m10w
ch51 c0h101 ch11
c4v2 c04402 c442
2y80 20y800 2y80
8j70 80j700 8j70
m9h7 m90h07 m9h7
3fp6 30fp06 3fp6
96vd 966v0d 96vd
2f9n 2f090n 2f9n
fw3y f0w30y fw3y
njxs n0jx0s njxs
3p41 3p0401 3p41
wty4 w0ty04 wty4
sndx s0nd0x sndx
26pv 206p0v 26pv
md4v m0d40v md4v
yhrr y0hr0r yhrr
2rtw 201r0w 21rw
f5ma f05m0a f5ma
53a8 50ja08 5ja8
kvs4 kvvs04 kvs4
mvkm m0vk0m mvkm
vpmk v0pmmk vpmk
py9j py990j py9j
mp7n m0p70n mp7n
r32t r0320t r32t
7pae 70pc0e 7pce
0p0w 00p00w 0p0w
f68j f0680j f68j
0enk 00e80k 0e8k
50fv 500f0v 50fv
0cmf 00cm0f 0cmf
jwfd j0w0fd jwfd
wsky w0sk0y wsky
n3at n03a0t n3at
6ycd 60y0cd 6ycd
k1st k01s0t k1st
ccn5 c0cn05 ccn5
2n7y 20n70y 2n7y
161k 10670k 167k
t6x2 t06x02 t6x2
5vjv 5v0j0v 5vjv
8dwf 80dw0f 8dwf
ypcn y0p60n yp6n
52m4 520m04 52m4
r01r r0001r r01r
ce48 ce0408 ce48
j0y8 j00y08 j0

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


98vs
r7r4 r07r04 r7r4
mvj6 m0vj06 mvj6
v31x v0910x v91x
7c2m 70r20m 7r2m
0jvw 00jv0w 0jvw
hmpf h0mp0f hmpf
9wcs 90wc0s 9wcs
Accuracy: 0.750
Epoch  2


...

 v91x
3p41 3p0401 3p41
fjfr f0jf0r fjfr
v13a v0030a v03a
fsmk fs0m0k fsmk
96vd 966v0d 96vd
p0t4 p00t04 p0t4
5cp0 50cp00 5cp0
Accuracy: 0.785
Epoch  50


  0%|          | 0/1 [00:00<?, ?it/s]

2n7y 20n70y 2n7y
hmpf h0mp0f hmpf
md4v m0d40v md4v
esax e0sa0x esax
ca3h ca030h ca3h
ax80 a0x800 ax80
yt68 y0t608 yt68
j10m j1000m j10m
x5w4 x0sw04 xsw4
n0fn n00f0n n0fn
7x6f 70x60f 7x6f
e7v9 e07v09 e7v9
x4jt x04j0t x4jt
n3at n03a0t n3at
2rx9 20rx09 2rx9
f68j f0680j f68j
2511 205101 2511
3ffm 30fffm 3ffm
ev60 e0v800 ev80
3v26 30y206 3y26
hpst k0pt0t kptt
thd9 th0d09 thd9
5cp0 50cp00 5cp0
w5an w05a0n w5an
ppt5 p0pt05 ppt5
v31x v0910x v91x
p0t4 p00t04 p0t4
ytxy y0tx0y ytxy
xkyc x0kk0c xkkc
3r1a 3rr10a 3r1a
76xf 70sx0f 7sxf
16yh 106y0h 16yh
30vc 300v0c 30vc
1x0f 1x000f 1x0f
j7vs j00v0s j0vs
1t0j 10t00j 1t0j
ypcn y0p60n yp6n
h0vv h00v0v h0vv
vwn1 v0w0n7 vwn7
ce48 ce0408 ce48
6nvy 60nhvy 6nhvy
03t3 003t03 03t3
ykrh y0kn0h yknh
ydkd y0dk0d ydkd
83k8 80fk08 8fk8
p8hd p08h0d p8hd
ma8y m0a80y ma8y
njxs n0jx0s njxs
8y9a e0y90a ey9a
dsfj d0sf0j dsfj
nsms nssm0s nsms
e26k e0260k e26k
4ejs 40250s 425s
c902 c0r002 cr02
8a5s 80a60s 8a6s
3dm1 30dm01 3dm1
0ptf 00pt0f 0ptf
d6rh d06r0h d6rh
kywv k0yw0v k

100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

vk4h
nx3s n0x30s nx3s
8vwh 8v0w0h 8vwh
6wh3 60whh3 6wh3
2f9n 2f090n 2f9n
t6ph t60p0h t6ph
asek a0se0k asek
e767 e07607 e767
h2nc h02n0c h2nc
1hde 10hd0e 1hde
3x91 30x901 3x91
tyxj t0yx0j tyxj
r3tc r03t0c r3tc
ahs4 a0hs04 ahs4
trd2 t0rd02 trd2
77e0 700e00 70e0
vp01 v0p001 vp01
txjj t0xj0j txjj
0n0w 00n00w 0n0w
ewwf e0ww0f ewwf
Accuracy: 0.800


In [16]:
torch.save(model3, 'model3.pth')
!cp model3.pth ./drive/MyDrive/hw5/captcha-hacker/model3.pth 